# Day 6: Trash Compactor ---
```
After helping the Elves in the kitchen, you were taking a break and helping them re-enact a movie scene when you over-enthusiastically jumped into the garbage chute!

A brief fall later, you find yourself in a garbage smasher. Unfortunately, the door's been magnetically sealed.

As you try to find a way out, you are approached by a family of cephalopods! They're pretty sure they can get the door open, but it will take some time. While you wait, they're curious if you can help the youngest cephalopod with her math homework.

Cephalopod math doesn't look that different from normal math. The math worksheet (your puzzle input) consists of a list of problems; each problem has a group of numbers that need to be either added (+) or multiplied (*) together.

However, the problems are arranged a little strangely; they seem to be presented next to each other in a very long horizontal list. For example:

123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +  
Each problem's numbers are arranged vertically; at the bottom of the problem is the symbol for the operation that needs to be performed. Problems are separated by a full column of only spaces. The left/right alignment of numbers within each problem can be ignored.

So, this worksheet contains four problems:

123 * 45 * 6 = 33210
328 + 64 + 98 = 490
51 * 387 * 215 = 4243455
64 + 23 + 314 = 401
To check their work, cephalopod students are given the grand total of adding together all of the answers to the individual problems. In this worksheet, the grand total is 33210 + 490 + 4243455 + 401 = 4277556.

Of course, the actual worksheet is much wider. You'll need to make sure to unroll it completely so that you can read the problems clearly.

Solve the problems on the math worksheet. What is the grand total found by adding together all of the answers to the individual problems?
```

In [306]:
import re
import numpy as np
import sys
import itertools
import functools
from operator import add, mul
sys.path.append("..")

from common.utils import Day, read_lines
test_path = "test.txt"
data_path = "data.txt"

In [1]:
""" 
So, we read each line and do the operations, no?
"""

' \nSo, we read each line and do the operations, no?\n'

In [147]:
class Day6(Day):
    def __init__(self, path):
        super().__init__(path)
        self.lines = read_lines(path)
        self.numbers = self.lines2chararray()
        self.operations = self.get_operations()


    def lines2chararray(self) -> np.array:
        """
        interpret the first rows of the problem as an
        array of integers.
        """
        numeric_lines = []
        for l in self.lines:
            l = l.strip()
            nl = re.split(r" +", l)
            if nl[0].isdigit():
                numeric_lines.append(nl)
            
        num_array = np.array(numeric_lines).astype(int)
        return num_array 
    

    def get_operations(self):
        """
        read the operation line and store its operation:
        """
        op_line = self.lines[-1]

        return re.split(r" +", op_line.strip())

    def solve_part_1(self):
        """
        Read the array and compute the corresponding 
        operation for each column
        add all the results.
        """
        result = 0

        for i in range(0,self.numbers.shape[1], 1):
            numbers = self.numbers[:, i]
            operation = self.operations[i]

            if operation == "+":
                result += np.sum(numbers)
            elif operation == "*":
                result += np.prod(numbers)


        return result

    
day6 = Day6(test_path)
day6.solve_part_1()

np.int64(4277556)

In [148]:
day6 = Day6(data_path)
day6.solve_part_1()

np.int64(4805473544166)

# Part Two ---
```
The big cephalopods come back to check on how things are going. When they see that your grand total doesn't match the one expected by the worksheet, they realize they forgot to explain how to read cephalopod math.

Cephalopod math is written right-to-left in columns. Each number is given in its own column, with the most significant digit at the top and the least significant digit at the bottom. (Problems are still separated with a column consisting only of spaces, and the symbol at the bottom of the problem is still the operator to use.)

Here's the example worksheet again:

123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +  
Reading the problems right-to-left one column at a time, the problems are now quite different:

The rightmost problem is 4 + 431 + 623 = 1058
The second problem from the right is 175 * 581 * 32 = 3253600
The third problem from the right is 8 + 248 + 369 = 625
Finally, the leftmost problem is 356 * 24 * 1 = 8544
Now, the grand total is 1058 + 3253600 + 625 + 8544 = 3263827.

Solve the problems on the math worksheet again. What is the grand total found by adding together all of the answers to the individual problems?
```

In [ ]:
"""
I need to re-do the reading of the data, the rest should be easy.

Now, I need to account for empty spaces, each number is a combination of spaces and
at least one number, separated by another space:

- Split each row every 3 characters
- join all numeric columns into sets of n_cols * 3
- Transform the character matrix to get the correct numbers
- get the result for each set of operations
"""
from operator import add, mul

class Day6b(Day6):
    def __init__(self, path):
        super().__init__(path)
        self.lines = read_lines(path)

        self.operations = self.get_operations()
        self.numbers = self.read_squid_math()
    
    def chunk_lines(self, line: str, lenght = 4):
        """
        return a set of substrings of length 3 from the 
        complete line, separator will be ignored
        """

        return [
            line[0 + i: lenght + i][: (lenght-1)] 
            for i in range(0, len(line), lenght)
        ]
    
    def read_squid_math(self):
        """
        read the numbers in self.num_lines according to the weird rules.

        get a matrix
        
        """

        correct_numbers_strings = [
            "".join(row) for row in np.array(
                [[char for char in line ] for line in day6.lines[:-1]]
            ).T
        ]
        # clump groups of numbers into lists that will be added/multiplyed.
        correct_numbers = [
            list(element)
            for key, element in itertools.groupby(correct_numbers_strings, lambda x: not re.match(r"^\s+$", x))
            if key
        ]

        return correct_numbers
    
    def solve_part_2(self):
        """
        This was amistery to me, but once i got the list of numbers and the operators I should
        be able to get this result from a simple if statement for example
        this is just more fancy.

        itertools.reduce applies a function cumulatively to and iterable (like a list comprehension)
        so itertools.reduce(mult, ~[1,2,3,4,5,6]) = 6! 
        the same for itertools.reduce(add, ~[1,2,3,4]) = 10
        """
        ops = {"*": mul, "+": add}
        total = 0
        for operator, current_operands in zip(self.operations, self.numbers):
            total += functools.reduce(ops[operator], [int(x) for x in current_operands])
        total

        return total
        

day6 = Day6b(test_path)
day6.solve_part_2()

3263827

In [389]:
day6 = Day6b(data_path)
day6.solve_part_2()

8907730960817

## I found a solution for part 2 online
apparently my original idea od transposing works (?)

NOTE: test and data have different length of numbers, 
test has 3 digit numbers separated by whitespaces
data has 4 digit spaces
chunk_lines has to account for that